In [70]:
# Libraries
import os
import re
import json
import folium
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geonamescache import GeonamesCache
from urllib.request import urlopen


# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

api_key = "768d81e0c5c733e2a375488139b78bb0"

# getting lat and lon of random city
def getGeoResponse(city):
    pattern = r"^(?!.*\s)[\x00-\x7F]+$"
    city_name = str(city)
    if re.match(pattern, city_name):
        baseGeoUrl = f"http://api.openweathermap.org/geo/1.0/direct?q={city_name}&limit=1&appid={api_key}"
        try:
            geoResponse = urlopen(baseGeoUrl)
            return geoResponse
        except Exception as e:
            print(f"error while opening url: {str(e)}")
            return None
    


# returns list of random cities
def getRandomCities(num_cities):
    gc = GeonamesCache()
    cities = list(gc.get_cities().values())

    random_cities = random.sample(cities, num_cities)
    city_names = [city['name'] for city in random_cities]

    return city_names



def getCitiesCoords(num_cities):
    city_coords = []
    cities = getRandomCities(num_cities)
    for city in cities:
        city_response = getGeoResponse(city)
        if city_response is not None:
            try:
                city_dict = {}
                city_orig = json.loads(city_response.read())
                geo_df = pd.DataFrame(city_orig)
                city_dict["lat"] = geo_df.loc[0, "lat"]
                city_dict["lon"] = geo_df.loc[0, "lon"]
                city_coords.append(city_dict)
            except Exception as e:
                print(f"Error occured reading the response: {str(e)}")

    return city_coords



def getWeatherDataWithCoords(lat, lon):
    url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"
    # Send a request to the API and retrieve weather data for the coordinates
    response = urlopen(url)
    weather_data = json.loads(response.read())
    # Check if a city name is available in the response
    city_dict = {}
    if "name" in weather_data:
        city_dict["city_name"] = weather_data["name"]
        city_dict["lat"] = weather_data["coord"]["lat"]
        city_dict["lon"] = weather_data["coord"]["lon"]
        city_dict["curr_temp"] = weather_data["main"]["temp"]
        city_dict["max_temp"] = weather_data["main"]["temp_max"]
        city_dict["min_temp"] = weather_data["main"]["temp_min"]
        city_dict["sunrise"] = pd.to_datetime(weather_data["sys"]["sunrise"], unit="s")
        city_dict["sunset"] = pd.to_datetime(weather_data["sys"]["sunset"], unit="s")

    return city_dict
    


def getCitiesWeatherData():
    city_coords = getCitiesCoords(10)
    city_list = []
    for city in city_coords:
        lat = city["lat"]
        lon = city["lon"]
        city_data = getWeatherDataWithCoords(lat=lat, lon=lon) #returns a dict
        if len(city_data) > 0:
            city_list.append(city_data)

    return city_list


cities = getCitiesWeatherData()
df = pd.DataFrame(cities)

df


,Current Temperature,Max Temperature,Min Temperature,Sunrise,Sunset
0,286.21,287.38,284.59,2023-05-17 04:06:25,2023-05-17 19:47:35
